In [ ]:
import numpy as np
import torch
from tqdm import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import seaborn
import sys
seaborn.set()

import warnings
warnings.filterwarnings("ignore")

import reservoir
import reckernel
import kuramoto
%load_ext autoreload
%autoreload 2
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Erf activation function and arcsine kernel

In [ ]:
random_projection = 'gaussian'  # "gaussian" for RC, "structured" for SRC
redraw = False  # to choose if weights are redrawn at each iteration

n_rep = 40
input_len_range = np.arange(start=1, stop=10, step=1)
n_res_range = np.logspace(2, 4, num=20, dtype=int)
input_scale = 1  # scale of input in reservoir dynamics
res_scale = 1  # scale of reservoir in dynamics
n_input = 50 # number of time series, usually 50 is enough
input_dim = 100  # dimension of the time series, larger reduces the variance of the final result

loss = torch.nn.MSELoss()
RK = RecKernel(function='arcsin', res_scale=res_scale, input_scale=input_scale, memory_efficient=False)

error_asin = torch.zeros(len(input_len_range), len(n_res_range), n_rep)
for k in range(n_rep):
    torch.manual_seed(k)
    for i, n_res in tqdm(enumerate(n_res_range)):
        initial_states = torch.randn(n_input, n_res).to(device) / np.sqrt(n_res)
        initial_K = initial_states @ initial_states.T
        model = reservoir.ESN(input_dim, res_size=n_res, res_scale=res_scale, input_scale=input_scale, 
                              f='erf', random_projection=random_projection, seed=k, redraw=redraw)
        for j, input_len in (enumerate(input_len_range)):
            input_data = torch.randn(n_input, input_len, input_dim).to(device) / np.sqrt(input_dim)

            K = RK.forward(input_data, initial_K=initial_K)
            
            X_final = torch.zeros((n_input,n_res)).to(device)
            for n_in in range(n_input):
                X = model.forward(input_data[n_in,:,:].reshape(input_len,input_dim), initial_state=initial_state).to(device)
                X_final[n_in,:] = X[-1,:]
            K_hat = torch.matmul(X_final,X_final.t())
            
            error_asin[j, i, k] = loss(K_hat,K)

In [ ]:
# Plots
ncurves = 5
plt.loglog(n_res_range, torch.mean(error_asin, axis=2)[:ncurves, :].T)
plt.legend(input_len_range[:ncurves])
plt.title('Convergence of Reservoir Computing towards Recurrent Kernels')
plt.xlabel('Reservoir dimension')
plt.ylabel('Mean-square error');

In [ ]:
# Don't forget to change the file name
np.save('out/200521_conv_rc_asin_len1to10_nres1e2to1e4_in1res1', error_asin)

# Cosine activation function and RBF kernel

In [ ]:
random_projection = 'gaussian'  # "gaussian" for RC, "structured" for SRC
redraw = False  # to choose if weights are redrawn at each iteration

n_rep = 40
input_len_range = np.arange(start=1, stop=10, step=1)
n_res_range = np.logspace(2, 4, num=20, dtype=int)
input_scale = 1  # scale of input in reservoir dynamics
res_scale = 1  # scale of reservoir in dynamics
n_input = 50 # number of time series, usually 50 is enough
input_dim = 100  # dimension of the time series, larger reduces the variance of the final result

loss = torch.nn.MSELoss()
RK = RecKernel(function='rbf', res_scale=res_scale, input_scale=input_scale, memory_efficient=False)

error_rbf = torch.zeros(len(input_len_range), len(n_res_range), n_rep)
for k in range(n_rep):
    torch.manual_seed(k)
    for i, n_res in tqdm(enumerate(n_res_range)):
        initial_states = torch.randn(n_input, n_res).to(device) / np.sqrt(n_res)
        initial_K = initial_states @ initial_states.T
        model = reservoir.ESN(input_dim, res_size=n_res, res_scale=res_scale, input_scale=input_scale, 
                              f='cos_rbf', random_projection=random_projection, seed=k, redraw=redraw)
        for j, input_len in (enumerate(input_len_range)):
            input_data = torch.randn(n_input, input_len, input_dim).to(device) / np.sqrt(input_dim)

            K = RK.forward(input_data, initial_K=initial_K)
            
            X_final = torch.zeros((n_input,n_res)).to(device)
            for n_in in range(n_input):
                X = model.forward(input_data[n_in,:,:].reshape(input_len,input_dim), initial_state=initial_state).to(device)
                X_final[n_in,:] = X[-1,:]
            K_hat = torch.matmul(X_final,X_final.t())
            
            error_rbf[j, i, k] = loss(K_hat,K)

In [ ]:
# Plots
ncurves = 5
plt.loglog(n_res_range, torch.mean(error_rbf, axis=2)[:ncurves, :].T)
plt.legend(input_len_range[:ncurves])
plt.title('Convergence of Reservoir Computing towards Recurrent Kernels')
plt.xlabel('Reservoir dimension')
plt.ylabel('Mean-square error');

In [ ]:
# Don't forget to change the file name
np.save('out/200521_conv_rc_rbf_len1to10_nres1e2to1e4_in1res1', error_rbf)

# Heaviside function and arccos kernel

In [ ]:
random_projection = 'gaussian'  # "gaussian" for RC, "structured" for SRC
redraw = False  # to choose if weights are redrawn at each iteration

n_rep = 40
input_len_range = np.arange(start=1, stop=10, step=1)
n_res_range = np.logspace(2, 4, num=20, dtype=int)
input_scale = 1  # scale of input in reservoir dynamics
res_scale = 1  # scale of reservoir in dynamics
n_input = 50 # number of time series, usually 50 is enough
input_dim = 100  # dimension of the time series, larger reduces the variance of the final result

loss = torch.nn.MSELoss()
RK = RecKernel(function='acos heaviside', res_scale=res_scale, input_scale=input_scale, memory_efficient=False)

error_acos = torch.zeros(len(input_len_range), len(n_res_range), n_rep)
for k in range(n_rep):
    torch.manual_seed(k)
    for i, n_res in tqdm(enumerate(n_res_range)):
        initial_states = torch.randn(n_input, n_res).to(device) / np.sqrt(n_res)
        initial_K = initial_states @ initial_states.T
        model = reservoir.ESN(input_dim, res_size=n_res, res_scale=res_scale, input_scale=input_scale, 
                              f='heaviside', random_projection=random_projection, seed=k, redraw=redraw)
        for j, input_len in (enumerate(input_len_range)):
            input_data = torch.randn(n_input, input_len, input_dim).to(device) / np.sqrt(input_dim)

            K = RK.forward(input_data, initial_K=initial_K)
            
            X_final = torch.zeros((n_input,n_res)).to(device)
            for n_in in range(n_input):
                X = model.forward(input_data[n_in,:,:].reshape(input_len,input_dim), initial_state=initial_state).to(device)
                X_final[n_in,:] = X[-1,:]
            K_hat = torch.matmul(X_final,X_final.t())
            
            error_acos[j, i, k] = loss(K_hat,K)

In [ ]:
# Plots
ncurves = 5
plt.loglog(n_res_range, torch.mean(error_acos, axis=2)[:ncurves, :].T)
plt.legend(input_len_range[:ncurves])
plt.title('Convergence of Reservoir Computing towards Recurrent Kernels')
plt.xlabel('Reservoir dimension')
plt.ylabel('Mean-square error');

In [ ]:
# Don't forget to change the file name
np.save('out/200521_conv_rc_acos_len1to10_nres1e2to1e4_in1res1', error_acos)

# Sign function and arcsin kernel

In [ ]:
random_projection = 'gaussian'  # "gaussian" for RC, "structured" for SRC
redraw = False  # to choose if weights are redrawn at each iteration

n_rep = 40
input_len_range = np.arange(start=1, stop=10, step=1)
n_res_range = np.logspace(2, 4, num=20, dtype=int)
input_scale = 1  # scale of input in reservoir dynamics
res_scale = 1  # scale of reservoir in dynamics
n_input = 50 # number of time series, usually 50 is enough
input_dim = 100  # dimension of the time series, larger reduces the variance of the final result

loss = torch.nn.MSELoss()
RK = RecKernel(function='asin sign', res_scale=res_scale, input_scale=input_scale, memory_efficient=False)

error_asin2 = torch.zeros(len(input_len_range), len(n_res_range), n_rep)
for k in range(n_rep):
    torch.manual_seed(k)
    for i, n_res in tqdm(enumerate(n_res_range)):
        initial_states = torch.randn(n_input, n_res).to(device) / np.sqrt(n_res)
        initial_K = initial_states @ initial_states.T
        model = reservoir.ESN(input_dim, res_size=n_res, res_scale=res_scale, input_scale=input_scale, 
                              f='sign', random_projection=random_projection, seed=k, redraw=redraw)
        for j, input_len in (enumerate(input_len_range)):
            input_data = torch.randn(n_input, input_len, input_dim).to(device) / np.sqrt(input_dim)

            K = RK.forward(input_data, initial_K=initial_K)
            
            X_final = torch.zeros((n_input,n_res)).to(device)
            for n_in in range(n_input):
                X = model.forward(input_data[n_in,:,:].reshape(input_len,input_dim), initial_state=initial_state).to(device)
                X_final[n_in,:] = X[-1,:]
            K_hat = torch.matmul(X_final,X_final.t())
            
            error_asin2[j, i, k] = loss(K_hat,K)

In [ ]:
# Plots
ncurves = 5
plt.loglog(n_res_range, torch.mean(error_asin2, axis=2)[:ncurves, :].T)
plt.legend(input_len_range[:ncurves])
plt.title('Convergence of Reservoir Computing towards Recurrent Kernels')
plt.xlabel('Reservoir dimension')
plt.ylabel('Mean-square error');

In [ ]:
# Don't forget to change the file name
np.save('out/200521_conv_rc_acos_len1to10_nres1e2to1e4_in1res1', error_asin2)